In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
import numpy as np
from sklearn.svm import SVC, SVR
from pathlib import Path
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import roc_auc_score, r2_score, f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

from src.data.splits import via11_splits

from tqdm import tqdm

In [3]:
sst_embdeds = pd.read_pickle('/mrhome/vladyslavz/git/central-sulcus-analysis/data/via11/nobackup/contrastive_embeddings/via11-monai-BasicUnet-2x-segmentDice+contrastive/sst_embeds.pkl')
# sst_embdeds.encoder_embed = None

In [4]:
metrics_paths = Path('/mnt/projects/VIA_Vlad/nobackup/MP2RAGE_FS7_1_1/mindboggle').glob('via*')
metrics_paths = [x/'tables/left_cortical_surface/sulcus_shapes.csv' for x in metrics_paths]

In [5]:
# features2analyze = ['area', 'travel depth: median', 'geodesic depth: median', 
#                     'mean curvature: median',  'freesurfer curvature: MAD',
#                     'freesurfer convexity (sulc): median', 'freesurfer thickness: median']

In [6]:
# def extract_features(subj_path):
#     metrics_df = pd.read_csv(subj_path)
#     metrics_df = metrics_df.set_index('name')
#     features = dict(metrics_df.loc['central sulcus', :])
#     features['caseid'] = 'sub-' + subj_path.parent.parent.parent.name
#     return features
# features_df = []
# for subj_path in tqdm(metrics_paths):
#     features_df.append(extract_features(subj_path))
# features_df = pd.DataFrame(features_df).set_index('caseid')
# features_df

In [7]:
via_qc = pd.read_excel('/mnt/projects/VIA_Vlad/nobackup/QA_centralSulcus_lkj.xlsx')
via_demo = pd.read_excel('/mnt/projects/VIA_Vlad/nobackup/VIA11_fmriflanker_info_database_2021jan25.xlsx')
via_demo = via_demo[~via_demo.site.isna()]
via_demo['caseid'] = ['sub-via'+str(i).zfill(3) for i in via_demo['via_id']]
via_demo = via_demo.set_index('caseid')

viaid2qc = {1:[], 2:[], 3:[], 999:[]}
for i, row in via_qc.iterrows():
    viaid2qc[row['vis_QA']].append(row['subjects'])
    
via_demo = via_demo.loc[np.concatenate(list(viaid2qc.values()))]

In [8]:
features_df = via_demo[['group', 'sex', 'handedness', 'cbcl_total', 'cbcl_external', 'cbcl_internal', 'cgas', 'any_diagnosis']]

In [9]:
merged_df = sst_embdeds.merge(features_df, left_on='caseid', right_index=True)

In [10]:
features_df.handedness.value_counts()

3.0    258
1.0     26
2.0     19
Name: handedness, dtype: int64

In [11]:
# val_case_ids = np.random.choice(merged_df.caseid.unique(), 20, replace=False)

In [12]:
val_case_ids = np.random.choice(via11_splits['validation'], 20, replace=False)

In [13]:
REGR_feat = ['cbcl_total', 'cbcl_external',
       'cbcl_internal', 'cgas']

CATEG = ['group', 'sex', 'handedness', 'any_diagnosis']

In [14]:
targ= 'mlp_embed'
for f in REGR_feat:
    clean_df = merged_df[~merged_df[f].isna()]
    train_df = clean_df[~clean_df.caseid.isin(val_case_ids)]
    test_df = clean_df[clean_df.caseid.isin(val_case_ids)]

    train_y = train_df[f].values
    train_X = np.vstack(train_df[targ].values)

    test_y = test_df[f].values
    test_X = np.vstack(test_df[targ].values)

    lr = SVR()
    train_pred_y = lr.fit(train_X, train_y).predict(train_X)
    test_pred_y = lr.predict(test_X)

    if r2_score(test_y, test_pred_y) > 0 or True:
        print('BINGO')
        print(f'For feature: {f}')
        print(f'R2 train: {r2_score(train_y, train_pred_y)}')
        print(f'R2 test: {r2_score(test_y, test_pred_y)}')
        print()
        print(f'MSE train: {np.mean((train_y - train_pred_y)**2)}')
        print(f'MSE test: {np.mean((test_y - test_pred_y)**2)}')
        print('____________________', '\n')

BINGO
For feature: cbcl_total
R2 train: 0.28605082392060144
R2 test: -0.30911648575534123

MSE train: 241.82114261419912
MSE test: 370.5923826739095
____________________ 

BINGO
For feature: cbcl_external
R2 train: -0.023820727905786487
R2 test: -0.3385424175618994

MSE train: 24.002327779219844
MSE test: 31.17580234587383
____________________ 

BINGO
For feature: cbcl_internal
R2 train: 0.1359292965595037
R2 test: -0.174084805591473

MSE train: 36.69104309724179
MSE test: 52.4402864414319
____________________ 

BINGO
For feature: cgas
R2 train: 0.33525355784063293
R2 test: -0.5299758707003428

MSE train: 163.16811904228138
MSE test: 324.1100884491607
____________________ 



In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

In [22]:
val = clean_df.caseid.isin(via11_splits['validation'])

In [23]:
targ= 'mlp_embed'
for f in CATEG:
    clean_df = merged_df[~merged_df[f].isna()]
    # train_df = clean_df[(~clean_df.caseid.isin(val_case_ids)) & val]
    train_df = clean_df[(~clean_df.caseid.isin(val_case_ids))]
    test_df = clean_df[clean_df.caseid.isin(val_case_ids)]

    train_y = train_df[f].values
    train_X = np.vstack(train_df[targ].values)

    test_y = test_df[f].values
    test_X = np.vstack(test_df[targ].values)

    lr = SVC(probability=True, C=0.1)
    train_pred_y = lr.fit(train_X, train_y).predict(train_X)
    test_pred_y = lr.predict(test_X)
    
    cat = list(np.unique(train_y))
    train_y_ohe = OneHotEncoder(categories=cat).fit_transform(train_y.reshape(-1, 1)).toarray()
    test_y_ohe = OneHotEncoder(categories=cat).fit_transform(test_y.reshape(-1, 1)).toarray()
    
    train_pred_y_proba = lr.fit(train_X, train_y).predict_proba(train_X)
    test_pred_y_proba = lr.predict_proba(test_X)
    

    if  True:
        # print('BINGO')
        print(f'For feature: {f}')
        print(f'ROC-AUC train: {roc_auc_score(train_y_ohe, train_pred_y_proba, multi_class="ovr")}')
        print(f'ROC-AUC test: {roc_auc_score(test_y_ohe, test_pred_y_proba, multi_class="ovr")}')
        print()
        print(f'F1 train: {f1_score(train_y, train_pred_y, average="macro")}')
        print(f'F1 test: {f1_score(test_y, test_pred_y, average="macro")}')
        print()
        print(f'balanced_accuracy_score train: {balanced_accuracy_score(train_y, train_pred_y)}')
        print(f'balanced_accuracy_score test: {balanced_accuracy_score(test_y, test_pred_y)}')
        
        print('____________________', '\n')

TypeError: LabelBinarizer.fit_transform() takes 2 positional arguments but 3 were given